In [1]:
import html
from bs4 import BeautifulSoup
from glob import glob
from itertools import chain
from pathlib import Path
import pandas as pd

## Arabic - Hebrew

In [2]:
cards = []
KOL_HA = ['האפשריות נכונות', 'האפשרויות נכונות', 'כל התשובות', 'התשובות נכונות']
for file in sorted(glob("data/class_pages/*.txt")):
    if file.startswith('custom_'):
        continue
    print(file)
    with open(file, 'r', encoding='utf8') as f:
        html_content = f.read()
    # Use BeautifulSoup to parse the page
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all divs with the class 'problems-wrapper'
    problems_wrappers = soup.find_all('div', class_='problems-wrapper')

    # Loop over each div and do something with it
    for problems_wrapper in problems_wrappers:
        html_encoded = problems_wrapper.get("data-content")
        if "תרגמו לעברית" in html_encoded and "ترجمو للعبري" in html_encoded or ('77.txt' in file):
            html_decoded = html.unescape(html_encoded)
            prob_soup = BeautifulSoup(html_decoded, 'html.parser')
            if '77.txt' in file and not ('בחלק מהסעיפים' in html_decoded or 'הקשיבו' in html_decoded):
                continue
            problems = prob_soup.find_all('div', class_='problem')
            for problem in problems:
                #print(problem.prettify())
                audio = problem.find_all('source', type="audio/mp3")
                if "מה נאמר בהקלטה" in problem.prettify() or "הקשיבו ובחרו במשפט שנאמר" in problem.prettify():
                    print("MA NEEMAR")
                    questions = list(problem.find_all('div', class_='wrapper-problem-response'))
                    for term, answer, aud in zip(questions[::2], questions[1::2], audio):
                        audio_file_path = aud.get('src')
                        print(audio_file_path)
                        try:
                            term = term.find('label', class_=lambda x: 'correct' in x).text.strip()
                            print(term)
                            answer = answer.find('label', class_=lambda x: 'correct' in x).text.strip()
                            print(answer)
                            print('')
                        except:
                            print(question)
                        cards.append((audio_file_path, term, answer))
                elif problem.find('legend') is not None or '57' in file:
                    print("LEGEND")
                    questions = problem.find_all('div', class_='wrapper-problem-response')
#                     if len(questions)!= len(audio):
#                         print('ADDING PARAGRAPHS')
#                         paragraphs = problem.find_all('p')
                        
#                         # Combine the lists
#                         questions = sorted(questions + paragraphs, key=lambda x: x.sourceline)
#                     print(len(questions), len(audio))
                    for question, aud in zip(questions, audio):
                        audio_file_path = aud.get('src')
                        print(audio_file_path)
                        try:
                            term = question.find('legend')
                            if term is not None:
                                term = term.text.strip()
                            elif question.find('label', class_='problem-group-label') is not None:
                                term = question.find('label', class_='problem-group-label').text
                            else:
                                p = None
                                for sibling in question.previous_siblings:
                                    # Check if the sibling is a p element
                                    if sibling.name == 'p':
                                        # If it is, we found the first preceding p for this div
                                        p = sibling
                                        term = p.text
                                        break  # We only need the first p element above the div
                                if p is None:
                                    raise ValueError
                            print(term)
                            answer = question.find('label', class_=lambda x: 'correct' in x)
                            if answer is not None:
                                answer = answer.text.strip()
                                if any([kh in answer for kh in KOL_HA]):
                                    all_answers = question.find_all('label')
                                    answer = ' | '.join([a.text.strip() for a in all_answers])
                            else:
                                answer = question.find('option', selected="true").text.strip()

                            print(answer)
                            print('')
                            cards.append((audio_file_path, term, answer))
                        except:
                            print(problem)
        elif "הקשיבו ובחרו במשפט שנאמר" in html_encoded:
            html_decoded = html.unescape(html_encoded)
            prob_soup = BeautifulSoup(html_decoded, 'html.parser')
            problems = prob_soup.find_all('div', class_='problem')
            for problem in problems:
                #print(problem.prettify())
                audio = problem.find_all('source', type="audio/mp3")
                print("HAKSHIVU")
                questions = list(problem.find_all('div', class_='wrapper-problem-response'))
                for term, answer, aud in zip(questions[::2], questions[1::2], audio):
                    audio_file_path = aud.get('src')
                    print(audio_file_path)
                    try:
                        term = term.find('label', class_=lambda x: 'correct' in x).text.strip()
                        print(term)
                        answer = answer.find('label', class_=lambda x: 'correct' in x).text.strip()
                        print(answer)
                        print('')
                    except:
                        print(question)
                    cards.append((audio_file_path, term, answer))

class_pages/12.txt
LEGEND
https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les1/Les1Unit2Rec3f.mp3
אִלְמַטַאר כְּבִּיר
שדה התעופה גדול

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les1/Les1Unit2Rec4f.mp3
הַאדַא (א)לְוַלַד אַַשְְטַר מִן אַח'וּה
הילד הזה יותר חכם מאח שלו

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les1/Les1Unit2Rec5f.mp3
אַכְּּּּל אִמִّי אַזְכַּא מִן אַכְּּּּל אִמַّכּ
האוכל של אימא שלי יותר טעים מהאוכל של אימא שלך

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les1/Les1Unit2Rec6f.mp3
בַּנְטַלוֹן אַבּוּי אַעְ'לַא מִן בַּנְטַלוֹנִי
המכנסיים של אבא שלי יותר יקרים מהמכנסיים שלי

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les1/Les1Unit2Rec8f.mp3
אַחְמַד
סתם השם אחמד

class_pages/13.txt
LEGEND
https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les1/Les1Unit3Rec15m.mp3
חַדִיקַה
גן

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les1/Les1Unit3Rec16f.mp3
זַאכִּי
טעים

https://madrasa-voice.s3.eu-west-2.am

In [3]:
len(cards)

337

In [4]:
filtered_cards = []
BLACKLIST = ['בחלק מהסעיפים תופיע הקלטה בלבד, ובאחרים תוכלו להיעזר בטקסט בערבית']
for audio, arabic, hebrew in cards:
    if arabic not in BLACKLIST:
        filtered_cards.append((audio, arabic, hebrew))
cards = filtered_cards
len(cards) 

332

In [5]:
fixes = { 
    'hebrew': { 
        'תביא את האוכל הזה כדי שאוכל ביחד': 'תביא את האוכל הזה כדי שנאכל ביחד' 
    },
    'arabic' : {}
}
print(fixes['hebrew'].keys())
fixed_cards = []
for aud, arabic, hebrew in cards:
    if hebrew.strip() in fixes['hebrew']:
        print(f'changed hebrew card')
        print(f'from {hebrew}')
        hebrew = fixes['hebrew'][hebrew.strip()]
        print(f'to {hebrew}')
    if arabic in fixes['arabic']:
        arabic = fixes['arabic'][arabic]
        print(f'changed arabic card')
    fixed_cards.append((aud, arabic, hebrew))
cards = fixed_cards

dict_keys(['תביא את האוכל הזה כדי שאוכל ביחד'])
changed hebrew card
from תביא את האוכל הזה כדי שאוכל ביחד
to תביא את האוכל הזה כדי שנאכל ביחד


In [6]:
for file in sorted(glob("data/class_pages/custom_*.txt")):
    if '74.txt' in file:
        print(file)
        with open(file, 'r', encoding='utf8') as f:
            html_content = f.read()
        # Use BeautifulSoup to parse the page
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all divs with the class 'problems-wrapper'
        problems_wrappers = soup.find_all('p')

        for problem in problems_wrappers:
            #print(problem.prettify())
            term = problem.text.strip()
            audio_file_path = problem.find('source', type="audio/mp3").get('src')
            answer = ''
            print(audio_file_path)
            print(term)
            print(answer)
            #print('\n')
            cards.append((audio_file_path, term, answer))


class_pages/custom_74.txt
https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les7/Les7Unit4rec1m.mp3
כְּתַאבִּי, צַאחִבְּתַכּ, אַח'וּנַא

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les7/Les7Unit4rec2f.mp3
וֵין אִלְכַּמְבְּיוּתֵר (א)לִגְ'דִיד תַבַּע אַח'וּכּ?

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les7/Les7Unit4rec3m.mp3
הַאדַא (א)לִכְּתַאבּ - תַבַּעִי, מִש תַבַּעַכּ

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les7/Les7Unit4rec4f.mp3
אִ(ל)סַיַّארַה תַבַּעְתִי חַ'רְבַּאנֵה

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les7/Les7Unit4rec5.mp3
לַמִין הַאדַא? -הַאדַא לַהִבַּה

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les7/Les7Unit4rec6.mp3
הַאדַא אִלַכּ? - לַא, הַאדַא אִלוֹ

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les7/Les7Unit4rec7m.mp3
הַאדִי (אל)צّוּרה אִלַכּ

https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les7/Les7Unit4rec8f.mp3
בֵּיתִי גַ'מְבּ בֵּית אִ(ל)רַّאִיס

https://madrasa-v

In [7]:
vocab_df = pd.read_csv('data/vocab_with_audio.csv')
vocab_df['unit'] = vocab_df['unit'].fillna(9).astype(int)
vocab_df['lesson'] = vocab_df['lesson'].fillna(9).astype(int)

vocab_df.head()

,arabic,hebrew,lesson,unit,audio,course
0,הַאת!,תן!,1,4,https://madrasa-voice.s3.eu-west-2.amazonaws.c...,ממשיכים
1,הַ-(א)לְקַדֵّה,עד כדי כך,1,6,https://madrasa-voice.s3.eu-west-2.amazonaws.c...,ממשיכים
2,שְוַי שְוַי,"לאט, לאט לאט",1,6,https://madrasa-voice.s3.eu-west-2.amazonaws.c...,ממשיכים
3,יַמַّא,אמא (פנייה),1,6,https://madrasa-voice.s3.eu-west-2.amazonaws.c...,ממשיכים
4,אַהְלַן וּסַהְלַן,ברוך הבא,1,7,https://madrasa-voice.s3.eu-west-2.amazonaws.c...,ממשיכים


In [8]:
vocab_df.shape

(477, 6)

In [9]:
vocab_df.groupby('course').arabic.nunique()

course
ממשיכים    475
Name: arabic, dtype: int64

In [10]:
vocab_df.audio.iat[-1]

'https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Words/Course2Word(477).mp3'

In [11]:
for _, s in vocab_df.sort_values(['lesson', 'unit']).iterrows():
    remote_file_name = s.audio.split('/')[-1]
    local_file_name = f"course2_Les{s.lesson}_Les{s.lesson}Unit{s.unit}_{remote_file_name}"
    card = ((s.audio, local_file_name), s.arabic ,s.hebrew)
    cards.append(card)

In [12]:
len(cards)

819

In [13]:
len(cards)

819

In [14]:
cards[0][0]

'https://madrasa-voice.s3.eu-west-2.amazonaws.com/course2/Les1/Les1Unit2Rec3f.mp3'

In [15]:
import requests
import os
import shutil
from tqdm import tqdm
s3_dir = Path('data/s3')
for afp, term, answer in tqdm(cards):
    if type(afp)==str:
        save_path = s3_dir / afp.split('.com/')[1]
        cpy = True
    else:
        afp, save_path = (afp[0], s3_dir / afp[1])
        cpy = False

    if cpy and os.path.exists(save_path):
        new_save_path = s3_dir / afp.split('.com/')[1].replace('/','_')
        shutil.copy(save_path, new_save_path)    
    elif not os.path.exists(save_path):
        file = requests.get(afp, stream=True)
        parent_dir = save_path.parent
        parent_dir.mkdir(parents=True, exist_ok=True)
        with open(save_path, 'wb') as f:
            f.write(file.content)
        

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 819/819 [00:00<00:00, 2305.26it/s]


In [16]:
import random
import genanki
random.seed(64)

# model_id = random.randrange(1 << 30, 1 << 31)
# deck_id = random.randrange(1 << 30, 1 << 31)
model_id = 2096283916
deck_id = 1341857280

print(model_id)
print(deck_id)


my_model = genanki.Model(
  model_id,
  'Madrasa',
  fields=[
    {'name': 'Recording'},
    {'name': 'Arabic', 'rtl': True, 'html': True},
    {'name': 'Hebrew', 'rtl': True, 'html': True},
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Recording}}',
      'afmt': '{{FrontSide}}'
              '<br><hr id="answer"><br>'
              '<div style="text-align:center;font-size:30px">'
              '<div style="color:#00055f"><span dir="rtl">&rlm;{{Arabic}}&rlm;</span></div>'
              '<br><br>'
              '<span dir="rtl">&rlm;{{Hebrew}}&rlm;</span>'
              '</div>',
    },
  ],
    css='''.card {
 font-family: abraham;
 font-size: 30px;
 text-align: center
 
 }
 
 @font-face {
    font-family: abraham;
    src: url('_Abraham-Light.otf')
}
''')

2096283916
1341857280


In [17]:
my_deck = genanki.Deck(
  deck_id,
  'Madrasa Recordings')


In [18]:
cards = sorted(cards, key=lambda x: x[0].split('.com/')[1].replace('/','_') if type(x[0])==str else x[0][1])

In [19]:
len(cards)

819

In [20]:
notes = []
for afp, arabic, hebrew in cards:
    if type(afp)==str:
        local_audio_path = afp.split('.com/')[1].replace('/','_')
    else:
        local_audio_path = afp[1]
    my_note = genanki.Note(
        model=my_model,
        fields=[f'[sound:{local_audio_path}]',
                html.escape(arabic),
                html.escape(hebrew)])
    my_deck.add_note(my_note)

In [21]:
str(s3_dir)

's3'

In [22]:
my_package = genanki.Package(my_deck)
my_package.media_files = [str(s3_dir) + '/' + file for file in os.listdir(s3_dir)]

In [23]:
my_package.write_to_file('data/madrasa_recordings.apkg')

## Hebrew - Arabic

In [24]:
ha_cards = []
for file in sorted(glob("data/class_pages/*.txt")):
    if file.startswith('custom_'):
        continue
    print(file)
    with open(file, 'r', encoding='utf8') as f:
        html_content = f.read()
    # Use BeautifulSoup to parse the page
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all divs with the class 'problems-wrapper'
    problems_wrappers = soup.find_all('div', class_='problems-wrapper')

    # Loop over each div and do something with it
    for problems_wrapper in problems_wrappers:
        html_encoded = problems_wrapper.get("data-content")
        if "תרגמו לערבית" in html_encoded and "ترجمو للعربي" in html_encoded:
            html_decoded = html.unescape(html_encoded)
            prob_soup = BeautifulSoup(html_decoded, 'html.parser')
            problems = prob_soup.find_all('div', class_='problem')
            for problem in problems:
                #print(problem.prettify())
                if True: #problem.find('legend') is not None:
                    print("LEGEND")
                    questions = problem.find_all('div', class_='wrapper-problem-response')
#                     if len(questions)!= len(audio):
#                         print('ADDING PARAGRAPHS')
#                         paragraphs = problem.find_all('p')
                        
#                         # Combine the lists
#                         questions = sorted(questions + paragraphs, key=lambda x: x.sourceline)
#                     print(len(questions), len(audio))
                    for question in questions:
                        try:
                            term = question.find('legend')
                            if term is not None:
                                term = term.text.strip()
                            elif question.find('label', class_='problem-group-label') is not None:
                                term = question.find('label', class_='problem-group-label').text
                            else:
                                p = None
                                for sibling in question.previous_siblings:
                                    # Check if the sibling is a p element
                                    if sibling.name == 'p':
                                        # If it is, we found the first preceding p for this div
                                        p = sibling
                                        term = p.text
                                        break  # We only need the first p element above the div
                                if p is None:
                                    raise ValueError
                            print(term)
                            answer = question.find('label', class_=lambda x: 'correct' in x)
                            if answer is not None:
                                answer = answer.text.strip()
                            else:
                                answer = question.find('option', selected="true").text.strip()
                            print(answer)
                            print('')
                            ha_cards.append((term, answer))
                        except:
                            print(problem)

class_pages/12.txt
LEGEND
אני גבוה ממך
אַנַא אַטְוַל מִנַّכּ

החנות חדשה אבל החנות הזאת יותר חדשה
אִלְמַחַלّ גְ'דִיד בַּסّ הַאדַא (א)לְמַחַלّ אַגְ'דַד

אני לא יותר קצר מהם
אַנַא מִש אַקְצַר מִנְהֹם

אני לא בטוח
מִש מִתְאַכֵּّד

class_pages/13.txt
LEGEND
אני אוהב אותך יותר ממנו
אַנַא בַּחֵבַּّכּ אַכְּתַר מִנّוֹ

מי הבת הכי חכמה במשפחה?
מִין אַַשְְטַר בִּנְת בִּ(א)לְעֵילֵה?

המים שלי יותר קרים מהמים שלך
מַיְّתִי אַבְּרַד מִן מַיְّתַכּ

class_pages/14.txt
LEGEND
אני רוצה את החולצה הוורודה
בִּדִّי (א)לִבְּלוּזֵה (אל)זַّהְרִיֵה

הילדים מביאים את הכיסאות, ואנחנו צריכים להביא את השולחנות האדומים
(א)לִוְלַאד בִּ(י)גִ'יבּוּ (א)לְכַּרַאסִי, וּאִחְנַא לַאזֵם נְגִ'יבּ אִ(ל)טַّאוְלַאת אִלְחֻמֻר

זה לא הדבר הכי פשוט בעולם
הַאדַא מִש אַבְּסַט אִשִי בִּ(א)לְעַאלַם

הבעיה שלך יותר חשובה מהבעיה שלנו
מֻשְכִּלְתַכּ אַהַםّ מִן מֻשְכִּלְתְנַא

הרופאה אומרת שאני צריך לעשות יותר ספורט
אִ(ל)דַّכְּתוֹרַה בִּתְקוּל אִנّוֹ לַאזֵם אַלְעַבּ רִיַאצַ'ה אַכְּתַר

class_pages/15.txt
LEGEND
אני לא רוצה לשבת כאן! אני לא 

In [25]:
months = '''ינואר - כַּאנוּן אֵ(ל)תַّאנִי
פברואר - שֻבַּאט
מרץ - אַאדַ׳אר
אפריל - נִיסַאן
מאי - אַיַّאר
יוני - חֻזֵירַאן
יולי - תַמّוּז
אוגוסט - אַאבּ
ספטמבר - אֵילוּל
אוקטובר - תִשְרִין אִלְאַוַّל
נובמבר - תִשְרִין אִ(ל)תַّאנִי
דצמבר - כַּאנוּן אִלְאַוַّל'''
for month in months.split('\n'):
    hebrew, arabic = [x.strip() for x in month.split('-')]
    print(hebrew, arabic)
    ha_cards.append((hebrew, arabic))

ינואר כַּאנוּן אֵ(ל)תַّאנִי
פברואר שֻבַּאט
מרץ אַאדַ׳אר
אפריל נִיסַאן
מאי אַיַّאר
יוני חֻזֵירַאן
יולי תַמّוּז
אוגוסט אַאבּ
ספטמבר אֵילוּל
אוקטובר תִשְרִין אִלְאַוַّל
נובמבר תִשְרִין אִ(ל)תַّאנִי
דצמבר כַּאנוּן אִלְאַוַّל


In [26]:
import random
import genanki
random.seed(15)

# model_id = random.randrange(1 << 30, 1 << 31)
# deck_id = random.randrange(1 << 30, 1 << 31)
model_id = 1522523693
deck_id =  1098770087

print(model_id)
print(deck_id)


my_model = genanki.Model(
  model_id,
  'Madrasa Hebrew-Arabic',
  fields=[
    {'name': 'Hebrew', 'rtl': True},
    {'name': 'Arabic', 'rtl': True},
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '<div><span dir="rtl">&rlm;{{Hebrew}}&rlm;</span></div>',
      'afmt': '{{FrontSide}}'
              '<br><hr id="answer"><br>'
              '<div style="color:#00055f"><span dir="rtl">&rlm;{{Arabic}}&rlm;</span></div>'
    },
  ],
    css='''.card {
 font-family: abraham;
 font-size: 30px;
 text-align: center
 
 }
 
 @font-face {
    font-family: abraham;
    src: url('_Abraham-Light.otf')
}
''')

1522523693
1098770087


In [27]:
my_deck = genanki.Deck(
  deck_id,
  'Madrasa Hebrew-Arabic')


In [28]:
#cards = sorted(cards, key=lambda x: x[0].split('.com/')[1].replace('/','_') if type(x)==str else x[0][1])

In [29]:
len(ha_cards)

285

In [30]:
for hebrew, arabic in ha_cards:
    my_note = genanki.Note(
        model=my_model,
        fields=[
                html.escape(hebrew),
                html.escape(arabic)])
    my_deck.add_note(my_note)

In [31]:
my_package = genanki.Package(my_deck)
my_package.media_files = ['data/s3/_Abraham-Light.otf', 'data/s3/_Abraham-Light.ttf']

In [32]:
my_package.write_to_file('data/madrasa_hebrew_arabic.apkg')